In [7]:
import pandas as pd
import json

In [8]:
import os
if 'R_HOME' not in os.environ:
  os.environ['R_HOME'] = '/usr/lib/R'

In [9]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [10]:
def convert_list_of_dicts_into_dataframe(dict_list):
  output = {}
  for keyname in dict_list[0].keys():
    output[keyname] = []
  for item in dict_list:
    for k,v in item.items():
      output[k].append(v)
  return pd.DataFrame.from_dict(output)

#print(convert_list_of_dicts_into_dataframe([
#  {'a': 3, 'b': 5},
#  {'a': 4, 'b': 6}
#]))

In [11]:
sessions_for_user_by_day_and_goal_for_all_users = json.load(open('browser_all_session_info_sept18.json'))

In [138]:
def compute_intensity(domains_and_sessions):
  #print(domains_and_sessions)
  number_of_sessions_total = 0
  number_of_sessions_where_intervention_was_seen = 0
  for domain_and_sessions in domains_and_sessions:
    domain = domain_and_sessions['domain']
    is_goal_enabled = domain_and_sessions['is_goal_enabled']
    is_goal_frequent = domain_and_sessions['is_goal_frequent']
    if not is_goal_enabled:
      continue
    for session_info in domain_and_sessions['session_info_list_for_domain']:
      number_of_sessions_total += 1
      if session_info['intervention_active'] != None:
        number_of_sessions_where_intervention_was_seen += 1
  if number_of_sessions_total == 0:
    return None
  return number_of_sessions_where_intervention_was_seen / number_of_sessions_total

def compute_intensity_other_goal_domains(domains_and_sessions, target_domain):
  number_of_sessions_total = 0
  number_of_sessions_where_intervention_was_seen = 0
  for domain_and_sessions in domains_and_sessions:
    domain = domain_and_sessions['domain']
    if domain == target_domain:
      continue
    is_goal_enabled = domain_and_sessions['is_goal_enabled']
    is_goal_frequent = domain_and_sessions['is_goal_frequent']
    if not is_goal_enabled:
      continue
    for session_info in domain_and_sessions['session_info_list_for_domain']:
      number_of_sessions_total += 1
      if session_info['intervention_active'] != None:
        number_of_sessions_where_intervention_was_seen += 1
  if number_of_sessions_total == 0:
    return None
  return number_of_sessions_where_intervention_was_seen / number_of_sessions_total

def test_compute_intensity():
  for sessions_for_user_by_day_and_goal in sessions_for_user_by_day_and_goal_for_all_users:
    user = sessions_for_user_by_day_and_goal['user']
    for day_domains_and_sessions in sessions_for_user_by_day_and_goal['days_domains_and_sessions']:
      #if day_domains_and_sessions['is_goal_enabled'] != True:
      #  continue
      if True not in ([x['is_goal_enabled'] for x in day_domains_and_sessions['domains_and_sessions']]):
        continue
      print(compute_intensity(day_domains_and_sessions['domains_and_sessions']))
      return
#test_compute_intensity()

In [57]:


def print_data_for_users():
  for sessions_for_user_by_day_and_goal in sessions_for_user_by_day_and_goal_for_all_users:
    user = sessions_for_user_by_day_and_goal['user']
    if user == '6216c587462fb5fa9ca9f0e9':
      print(sessions_for_user_by_day_and_goal)
#print_data_for_users()

In [140]:
def passes_filters(filter_funcs, item):
  for filter_func in filter_funcs:
    if not filter_func(item):
      return False
  return True

def make_dictlist_days(sessions_for_user_by_day_and_goal_for_all_users, item_filter_funcs=[], day_filter_funcs=[], user_filter_funcs=[]):
  output = []
  for sessions_for_user_by_day_and_goal in sessions_for_user_by_day_and_goal_for_all_users:
    user = sessions_for_user_by_day_and_goal['user']
    if not passes_filters(user_filter_funcs, sessions_for_user_by_day_and_goal):
      continue
    #epoch_list = [x['epoch'] for x in sessions_for_user_by_day_and_goal['days_domains_and_sessions']]
    #first_epoch_for_user = min(epoch_list)
    #last_epoch_for_user = max(epoch_list)
    for day_domains_and_sessions in sessions_for_user_by_day_and_goal['days_domains_and_sessions']:
      epoch = day_domains_and_sessions['epoch']
      #day_domains_and_sessions['days_since_install'] = epoch - first_epoch_for_user
      #day_domains_and_sessions['days_until_last'] = last_epoch_for_user - epoch
      if not passes_filters(day_filter_funcs, day_domains_and_sessions):
        continue
      intensity = compute_intensity(day_domains_and_sessions['domains_and_sessions'])
      for domain_and_sessions in day_domains_and_sessions['domains_and_sessions']:
        domain = domain_and_sessions['domain']
        intensity_other_goals = compute_intensity_other_goal_domains(day_domains_and_sessions['domains_and_sessions'], domain)
        time_on_domain_today = domain_and_sessions['time_on_domain_today']
        time_on_other_goal_domains_today = domain_and_sessions['time_on_other_goal_domains_today']
        is_goal_enabled = domain_and_sessions['is_goal_enabled']
        is_goal_frequent = domain_and_sessions['is_goal_frequent']
        newitem = {
          'user': user,
          'epoch': epoch,
          'domain': domain,
          'time_on_domain_today': time_on_domain_today,
          'time_on_other_goal_domains_today': time_on_other_goal_domains_today,
          'is_goal_enabled': is_goal_enabled,
          'is_goal_frequent': is_goal_frequent,
          'intensity': intensity,
          'intensity_other_goals': intensity_other_goals
        }
        if not passes_filters(item_filter_funcs, newitem):
          continue
        output.append(newitem)
  return output

def make_dataframe_days(sessions_for_user_by_day_and_goal_for_all_users, item_filter_funcs=[], day_filter_funcs=[], user_filter_funcs=[]):
  #sessions_for_user_by_day_and_goal_for_all_users = get_sessions_for_user_by_day_and_goal_for_all_users()
  output = make_dictlist_days(sessions_for_user_by_day_and_goal_for_all_users, item_filter_funcs, day_filter_funcs, user_filter_funcs)
  return convert_list_of_dicts_into_dataframe(output)

# item filter funcs
def enabled_goals_only(x):
  return x['is_goal_enabled'] == True

# day filter funcs
def not_first_day(x):
  return x['days_since_install'] != 0

def not_last_day(x):
  return x['days_until_last'] != 0

# user filter funcs
def at_least_two_days(x):
  return len(x['days_domains_and_sessions']) > 1

def at_least_two_weeks(x):
  #print(len(x))
  return len(x['days_domains_and_sessions']) > 13

#df = make_dataframe_days(sessions_for_user_by_day_and_goal_for_all_users, [enabled_goals_only], [], [])
df = make_dataframe_days(sessions_for_user_by_day_and_goal_for_all_users, [enabled_goals_only], [not_first_day, not_last_day], [])
#print(make_dataframe_days())

In [141]:
df.dtypes

user                                 object
epoch                                 int64
domain                               object
time_on_domain_today                  int64
time_on_other_goal_domains_today      int64
is_goal_enabled                        bool
is_goal_frequent                       bool
intensity                           float64
intensity_other_goals               float64
dtype: object

In [75]:
#ndata = df
#ndata = ndata[ndata['is_goal_enabled'] == True]
#ndata = ndata[ndata['is_goal_frequent'] != 'inconsistent']
#ndata.shape

In [142]:
df

,user,epoch,domain,time_on_domain_today,time_on_other_goal_domains_today,is_goal_enabled,is_goal_frequent,intensity,intensity_other_goals
0,c11e5f2d93f249b5083989b2,947,www.youtube.com,26,859,True,False,0.888889,1.0
1,c11e5f2d93f249b5083989b2,947,www.reddit.com,859,26,True,False,0.888889,0.0
2,c11e5f2d93f249b5083989b2,953,www.reddit.com,829,24669,True,True,0.227273,0.0
3,c11e5f2d93f249b5083989b2,953,www.youtube.com,24669,829,True,True,0.227273,1.0
4,c11e5f2d93f249b5083989b2,954,www.reddit.com,1394,0,True,True,1.000000,NaN
5,c11e5f2d93f249b5083989b2,955,www.youtube.com,949,1351,True,False,0.500000,1.0
6,c11e5f2d93f249b5083989b2,955,www.reddit.com,1351,949,True,False,0.500000,0.0
7,c11e5f2d93f249b5083989b2,956,www.youtube.com,2163,1540,True,False,0.500000,1.0
8,c11e5f2d93f249b5083989b2,956,www.reddit.com,1540,2163,True,False,0.500000,0.0
9,c11e5f2d93f249b5083989b2,957,www.youtube.com,15557,501,True,False,0.230769,1.0


In [143]:
%Rpush df

#filename = 'browser_time_on_domains_sept18.csv'
#df = pd.read_csv(filename)

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [144]:
%%R -w 5 -h 5 --units in -r 200

#install.packages('ez')
#install.packages('lme4')

library(lme4)
library(sjPlot)
library(lmerTest)
#library(ez)

In [145]:
%%R -w 5 -h 5 --units in -r 200

ndata = df
ndata$domain = as.factor(ndata$domain)
ndata$user = as.factor(ndata$user)
ndata$is_goal_enabled = as.factor(ndata$is_goal_enabled)
ndata$is_goal_frequent = as.factor(ndata$is_goal_frequent)
ndata$log_time_on_domain_today = log(ndata$time_on_domain_today)
ndata$log_time_on_other_goal_domains_today = log(ndata$time_on_other_goal_domains_today)
#ndata = subset(ndata, domain == 'www.facebook.com' | domain == 'www.youtube.com')
#ndata = subset(ndata, domain == 'www.facebook.com')
ndata$log_time_on_other_goal_domains_today[which(!is.finite(ndata$log_time_on_other_goal_domains_today))] <- 0

ndata_all = ndata
ndata_noyoutube = subset(ndata, domain != 'www.youtube.com')
ndata_youtube = subset(ndata, domain == 'www.youtube.com')
ndata_facebook = subset(ndata, domain == 'www.facebook.com')

#ndata = subset(ndata, time_on_other_goal_domains_today != 0)
#summary(ndata)

In [67]:
#%%R -w 5 -h 5 --units in -r 200

#ndata = df
#subset(ndata, time_on_domain_today == 105384637)

In [146]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_facebook
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + intensity_other_goals + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: log_time_on_domain_today ~ is_goal_frequent + intensity_other_goals +  
    (1 | user)
   Data: ndata

REML criterion at convergence: 24284.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.2269 -0.5053  0.0749  0.5948  5.0414 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 2.521    1.588   
 Residual             2.775    1.666   
Number of obs: 5956, groups:  user, 699

Fixed effects:
                        Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)              6.91680    0.08180  939.14602  84.557  < 2e-16 ***
is_goal_frequentTRUE    -0.13535    0.04965 5740.52163  -2.726  0.00643 ** 
intensity_other_goals   -0.38242    0.11735 1970.24302  -3.259  0.00114 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr) i__TRU
is_gl_fTRUE -0.289       
intnsty_th_ -0

In [150]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_all
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + intensity_other_goals + (1|user) + (1|domain), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: log_time_on_domain_today ~ is_goal_frequent + intensity_other_goals +  
    (1 | user) + (1 | domain)
   Data: ndata

REML criterion at convergence: 80770.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.1354 -0.5778  0.0845  0.6452  4.7939 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 1.387    1.178   
 domain   (Intercept) 1.482    1.217   
 Residual             4.867    2.206   
Number of obs: 17960, groups:  user, 789; domain, 12

Fixed effects:
                        Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)            6.753e+00  3.929e-01  6.893e+00   17.18 6.48e-07 ***
is_goal_frequentTRUE  -7.111e-02  3.469e-02  1.777e+04   -2.05   0.0404 *  
intensity_other_goals -9.142e-01  4.875e-02  1.667e+04  -18.75  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects

In [148]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_youtube
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + intensity_other_goals + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: log_time_on_domain_today ~ is_goal_frequent + intensity_other_goals +  
    (1 | user)
   Data: ndata

REML criterion at convergence: 25669.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9583 -0.5390  0.0793  0.6274  3.9644 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 3.093    1.759   
 Residual             4.710    2.170   
Number of obs: 5599, groups:  user, 708

Fixed effects:
                        Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)              7.69259    0.09956 1030.09582  77.267   <2e-16 ***
is_goal_frequentTRUE     0.04959    0.06667 5495.36872   0.744   0.4570    
intensity_other_goals   -0.35544    0.13916 1617.50454  -2.554   0.0107 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr) i__TRU
is_gl_fTRUE -0.322       
intnsty_th_ -0

In [151]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_noyoutube
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + intensity_other_goals + (1|user) + (1|domain), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: log_time_on_domain_today ~ is_goal_frequent + intensity_other_goals +  
    (1 | user) + (1 | domain)
   Data: ndata

REML criterion at convergence: 53450.7

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.2504 -0.5548  0.0772  0.6297  4.9756 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 1.708    1.307   
 domain   (Intercept) 1.464    1.210   
 Residual             3.962    1.990   
Number of obs: 12361, groups:  user, 788; domain, 11

Fixed effects:
                        Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)            6.513e+00  4.107e-01  7.173e+00  15.858 7.58e-07 ***
is_goal_frequentTRUE  -8.187e-02  3.855e-02  1.218e+04  -2.124   0.0337 *  
intensity_other_goals -5.866e-01  6.439e-02  8.749e+03  -9.111  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects

In [97]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_facebook
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: log_time_on_domain_today ~ is_goal_frequent + (1 | user)
   Data: ndata

REML criterion at convergence: 35987.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.1445 -0.5157  0.0856  0.6118  5.1972 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 2.601    1.613   
 Residual             2.896    1.702   
Number of obs: 8791, groups:  user, 825

Fixed effects:
                       Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)             6.64622    0.06520  884.83941 101.930  < 2e-16 ***
is_goal_frequentTRUE   -0.12654    0.04127 8506.73848  -3.066  0.00217 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
is_gl_fTRUE -0.302


In [98]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_youtube
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: log_time_on_domain_today ~ is_goal_frequent + (1 | user)
   Data: ndata

REML criterion at convergence: 34783.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9028 -0.5528  0.0919  0.6377  4.0817 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 3.115    1.765   
 Residual             5.005    2.237   
Number of obs: 7513, groups:  user, 841

Fixed effects:
                      Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)          7.489e+00  7.637e-02 1.025e+03  98.069   <2e-16 ***
is_goal_frequentTRUE 3.561e-02  5.947e-02 7.414e+03   0.599    0.549    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
is_gl_fTRUE -0.382


In [99]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_noyoutube
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: log_time_on_domain_today ~ is_goal_frequent + (1 | user) + (1 |  
    domain)
   Data: ndata

REML criterion at convergence: 69095.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4292 -0.5523  0.0737  0.6277  4.9370 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 1.876    1.370   
 domain   (Intercept) 1.357    1.165   
 Residual             3.872    1.968   
Number of obs: 16056, groups:  user, 939; domain, 12

Fixed effects:
                       Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)           6.238e+00  3.890e-01  8.390e+00  16.036 1.38e-07 ***
is_goal_frequentTRUE -8.102e-02  3.358e-02  1.581e+04  -2.413   0.0158 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
is_gl_fTRUE -0.045


In [103]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_all
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: log_time_on_domain_today ~ is_goal_frequent + (1 | user) + (1 |  
    domain)
   Data: ndata

REML criterion at convergence: 106276.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.1445 -0.5693  0.0874  0.6505  4.6140 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 1.590    1.261   
 domain   (Intercept) 1.325    1.151   
 Residual             4.906    2.215   
Number of obs: 23569, groups:  user, 1038; domain, 13

Fixed effects:
                       Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)           6.417e+00  3.673e-01  8.605e+00  17.470 5.09e-08 ***
is_goal_frequentTRUE -3.652e-02  3.062e-02  2.335e+04  -1.193    0.233    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
is_gl_fTRUE -0.043


In [92]:
%%R -w 5 -h 5 --units in -r 200

#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: log_time_on_domain_today ~ is_goal_frequent + (1 | user) + (1 |  
    domain)
   Data: ndata

REML criterion at convergence: 69095.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4292 -0.5523  0.0737  0.6277  4.9370 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 1.876    1.370   
 domain   (Intercept) 1.357    1.165   
 Residual             3.872    1.968   
Number of obs: 16056, groups:  user, 939; domain, 12

Fixed effects:
                       Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)           6.238e+00  3.890e-01  8.390e+00  16.036 1.38e-07 ***
is_goal_frequentTRUE -8.102e-02  3.358e-02  1.581e+04  -2.413   0.0158 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
is_gl_fTRUE -0.045


In [109]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_facebook
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user), data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user), data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) +  
    (1 | user)
   Data: ndata

REML criterion at convergence: 35821

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.1062 -0.5063  0.0796  0.6058  4.7474 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 2.637    1.624   
 Residual             2.831    1.683   
Number of obs: 8791, groups:  user, 825

Fixed effects:
                                       Estimate Std. Error         df t value
(Intercept)                             6.21322    0.07291 1296.06825  85.214
is_goal_frequentTRUE                   -0.13049    0.04083 8496.57377  -3.196
log_time_on_other_goal_domains_today    0.07949    0.00597 8755.05072  13.315
                                     Pr(>|t|)    
(Intercept)                            <2e-16 ***
is_goal_frequentTRUE                   0

In [108]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_youtube
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user), data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user), data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) +  
    (1 | user)
   Data: ndata

REML criterion at convergence: 34709.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.7840 -0.5492  0.0836  0.6324  4.0002 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 3.215    1.793   
 Residual             4.932    2.221   
Number of obs: 7513, groups:  user, 841

Fixed effects:
                                      Estimate Std. Error        df t value
(Intercept)                          6.995e+00  9.395e-02 1.853e+03  74.459
is_goal_frequentTRUE                 3.377e-02  5.911e-02 7.401e+03   0.571
log_time_on_other_goal_domains_today 9.030e-02  9.930e-03 6.813e+03   9.093
                                     Pr(>|t|)    
(Intercept)                            <2e-16 ***
is_goal_frequentTRUE                    0.568 

In [105]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_noyoutube
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user), data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user) + (1|domain), data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) +  
    (1 | user) + (1 | domain)
   Data: ndata

REML criterion at convergence: 69082.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.3643 -0.5533  0.0771  0.6267  4.8276 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 1.845    1.358   
 domain   (Intercept) 1.375    1.173   
 Residual             3.870    1.967   
Number of obs: 16056, groups:  user, 939; domain, 12

Fixed effects:
                                       Estimate Std. Error         df t value
(Intercept)                           6.094e+00  3.924e-01  8.544e+00  15.528
is_goal_frequentTRUE                 -8.059e-02  3.356e-02  1.582e+04  -2.401
log_time_on_other_goal_domains_today  2.446e-02  5.261e-03  1.504e+04   4.650
                                     Pr(>|t|)    
(Intercept)         

In [106]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_all
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user), data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user) + (1|domain), data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) +  
    (1 | user) + (1 | domain)
   Data: ndata

REML criterion at convergence: 106208.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.1734 -0.5670  0.0892  0.6521  4.7936 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 1.730    1.315   
 domain   (Intercept) 1.283    1.133   
 Residual             4.875    2.208   
Number of obs: 23569, groups:  user, 1038; domain, 13

Fixed effects:
                                       Estimate Std. Error         df t value
(Intercept)                           6.648e+00  3.631e-01  8.634e+00  18.306
is_goal_frequentTRUE                 -3.702e-02  3.055e-02  2.330e+04  -1.212
log_time_on_other_goal_domains_today -4.220e-02  4.734e-03  2.171e+04  -8.915
                                     Pr(>|t|)    
(Intercept)       


Call:
lm(formula = log_time_on_domain_today ~ is_goal_frequent, data = ndata)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.2747 -1.6770  0.0857  1.7400 11.2177 

Coefficients:
                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)           7.27473    0.02321 313.425   <2e-16 ***
is_goal_frequentTRUE -0.02937    0.03348  -0.877     0.38    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 2.568 on 23567 degrees of freedom
Multiple R-squared:  3.266e-05,	Adjusted R-squared:  -9.776e-06 
F-statistic: 0.7696 on 1 and 23567 DF,  p-value: 0.3803



In [115]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_facebook
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user), data = ndata)
results <- lm(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today), data = ndata)
summary(results)


Call:
lm(formula = log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today), 
    data = ndata)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.3467 -1.3906  0.1113  1.5099  8.5034 

Coefficients:
                                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)                           6.873814   0.045291 151.770  < 2e-16 ***
is_goal_frequentTRUE                 -0.053612   0.048424  -1.107    0.268    
log_time_on_other_goal_domains_today  0.039288   0.005548   7.081 1.54e-12 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 2.268 on 8788 degrees of freedom
Multiple R-squared:  0.005802,	Adjusted R-squared:  0.005576 
F-statistic: 25.64 on 2 and 8788 DF,  p-value: 7.851e-12



In [117]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_youtube
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user), data = ndata)
results <- lm(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today), data = ndata)
summary(results)


Call:
lm(formula = log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today), 
    data = ndata)

Residuals:
    Min      1Q  Median      3Q     Max 
-8.0753 -1.9383  0.2568  2.0285 10.4655 

Coefficients:
                                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)                           7.851724   0.065981 119.000   <2e-16 ***
is_goal_frequentTRUE                 -0.006515   0.064681  -0.101   0.9198    
log_time_on_other_goal_domains_today  0.020229   0.008288   2.441   0.0147 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 2.802 on 7510 degrees of freedom
Multiple R-squared:  0.0007948,	Adjusted R-squared:  0.0005287 
F-statistic: 2.987 on 2 and 7510 DF,  p-value: 0.05051



In [120]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_noyoutube
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user), data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|domain), data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) +  
    (1 | domain)
   Data: ndata

REML criterion at convergence: 72681.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.8197 -0.6213  0.0512  0.6621  4.1928 

Random effects:
 Groups   Name        Variance Std.Dev.
 domain   (Intercept) 1.492    1.222   
 Residual             5.395    2.323   
Number of obs: 16056, groups:  domain, 12

Fixed effects:
                                       Estimate Std. Error         df t value
(Intercept)                           5.986e+00  4.062e-01  6.541e+00  14.735
is_goal_frequentTRUE                 -6.734e-02  3.672e-02  1.604e+04  -1.834
log_time_on_other_goal_domains_today  4.575e-02  4.597e-03  1.605e+04   9.952
                                     Pr(>|t|)    
(Intercept)                          2.92e-06 ***
is_goal_frequentTRUE              

In [122]:
%%R -w 5 -h 5 --units in -r 200

ndata = ndata_all
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|user), data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) + (1|domain), data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
log_time_on_domain_today ~ is_goal_frequent + (log_time_on_other_goal_domains_today) +  
    (1 | domain)
   Data: ndata

REML criterion at convergence: 109874.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5592 -0.6332  0.0620  0.6794  4.2400 

Random effects:
 Groups   Name        Variance Std.Dev.
 domain   (Intercept) 1.462    1.209   
 Residual             6.180    2.486   
Number of obs: 23569, groups:  domain, 13

Fixed effects:
                                       Estimate Std. Error         df t value
(Intercept)                           6.241e+00  3.841e-01  7.393e+00  16.247
is_goal_frequentTRUE                 -4.752e-02  3.243e-02  2.356e+04  -1.465
log_time_on_other_goal_domains_today  3.782e-02  4.089e-03  2.356e+04   9.248
                                     Pr(>|t|)    
(Intercept)                          4.72e-07 ***
is_goal_frequentTRUE             